# Isentropic Flow

This Notebook illustrates the use of the *Isentropic Flow* class from **pyTurb**. An *Isentropic Flow* is considered to be *reversible* and to have no disipation, nor friction, maintaining a constant level of entropy. If the fluid undergoes such a transformation, then the energy balance of the fluid allows the transport of kinetic energy to thermal/reversible work.

- pyturb
    - gas_models
        - thermo_prop
        - PerfectIdealGas
        - SemiperfectIdealGas
        - GasMixture
        - IsentropicFlow
        
```python
from pyturb.gas_models import GasMixture
from pyturb.gas_models import PerfectIdealGas
from pyturb.gas_models import SemiperfectIdealGas
from pyturb.gas_models import IsentropicFlow
```

The `IsentropicFlow` can be initialized with pyTurb's `PerfectIdealGas` or `SemiperfectIdealGas` classes. Defining the model as perfect or smeiperfect indicates how the $c_p$, $c_v$ and $\gamma$ are treated:

- *PerfectIdealGas*: Ideal Equation of State ($pv=R_gT$) and constant $c_p$, $c_v$, $\gamma_g$
- *SemiperfectIdealGas*: Ideal Equation of State and $c_p\left(T\right)$, $c_v\left(T\right)$, $\gamma_g\left(T\right)$ as a function of temperature

From here, the content of the *IsentropicFlow* class includes:
- Sound speed
- Mach Number
- Stagnation relations:
    - Stagnation temperature
    - Stagnation pressure
    - Stagnation density
    - Impact pressure
    - Dynamic pressure
- Static relations:
    - Static temperature
    - Static pressure
    - Static density

---


### Isentropic flow examples
We will start with importing *air* as both pyTurb's *perfect* and *semi-perfect* gases and creating an Isentropic Gas object from both models:

In [1]:
from pyturb.gas_models import IsentropicFlow
from pyturb.gas_models import PerfectIdealGas
from pyturb.gas_models import SemiperfectIdealGas

In [2]:
perfect_air = PerfectIdealGas('Air')
semiperfect_air = SemiperfectIdealGas('Air')

perf_air_isentflow = IsentropicFlow(perfect_air)
semiperf_air_isentflow = IsentropicFlow(semiperfect_air)

We will also get the local temperature, pressure and density of the air at $5000m$ from the Standard Atmosphere (ISA) from *pyTurb*:

In [3]:
from pyturb.gas_models import isa

h0 = 5000 #m
T0 = isa.temperature_isa(h0)
p0 = isa.pressure_isa(h0)

rho0 = p0/perf_air_isentflow.fluid.Rg/T0

print("At {0:4.0f}m above SL:  T0={1:6.2f} K;  p0={2:6.0f} Pa;  rho0={3:6.4f} kg/m^3".format(h0, T0, p0, rho0))

At 5000m above SL:  T0=255.65 K;  p0= 54020 Pa;  rho0=0.7361 kg/m^3


Note that the `PerfectIdealGas`or the `SemiperfectIdealGas` ojects are copied to `IsentropicFlow`. If, for example, the gas constant is needed, it can be accessed directly from the Isentropic Gas object:

```python
from pyturb.gas_models import PerfectIdealGas
from pyturb.gas_models import IsentropicFlow

air = PerfectIdealGas("air")
isent_flow = IsentropicFlow(air)
isent_flow.fluid.Rg
```

> `fluid` is a copy of the pyTurb gas model used to create the  `IsentropicFlow` object.

---

### Sound Speed and Mach Number

Sound speed indicates the ratio at wich a pressure perturbation can travel within the fluid. It is related to the specific energy that is carried by a pressure wave:

$$\frac{\partial p}{\partial \rho}\bigg \vert_s = \gamma \frac{p}{\rho} = \gamma R_g T = a^2 $$

Thus the sound speed is:

$$c=\sqrt{\frac{\partial p}{\partial \rho}\bigg \vert_s } = \sqrt{\gamma R_g T}$$

In *pyTurb* the sound speed can be calculated with *sound_speed(static_temperature)* where `static_temperature` is the local flow temperature at which the sound speed is desired. Note that in case of a *semi-perfect* gas, pyturb calculate the corresponding value of $\gamma\left(T\right)$.

For example, for the local temperature at $5000m$ above sea level:

In [4]:
a0_pf = perf_air_isentflow.sound_speed(T0)
a0_spf = semiperf_air_isentflow.sound_speed(T0)

print('Perfect Gas:       a0={0:6.2f}m/s'.format(a0_pf))
print('Semi-perfect Gas:  a0={0:6.2f}m/s'.format(a0_spf))

Perfect Gas:       a0=320.53m/s
Semi-perfect Gas:  a0=320.62m/s


The Mach number is calculated as the coefficient between the local speed and the sound speed of the flow:

$$M=\frac{v}{a}$$

If we consider a local velocity of $200\frac{m}{s}$:

In [5]:
v0 = 200 #m/s
M0_pf = perf_air_isentflow.mach_number(v0,T0)
M0_spf = semiperf_air_isentflow.mach_number(v0,T0)
print('Perfect Gas:       M0={0:6.4f}m/s'.format(M0_pf))
print('Semi-perfect Gas:  M0={0:6.4f}m/s'.format(M0_spf))

Perfect Gas:       M0=0.6240m/s
Semi-perfect Gas:  M0=0.6238m/s


### Stagnation temperature:

The stagnation temperature can be calculated as the static temperature plus the kinetic energy of the fluid (`stag_temp_from_vel`):

$$ T_t = T + \frac{1}{2}\frac{v^2}{c_p}$$

This relation can be rearrenged with the Mach Number (`stagnation_static_relation`):

$$ \frac{T_t}{T} = 1 + \frac{\gamma-1}{2}M^2$$



In [6]:
# Perfect Gas:
T0t_vel_pf = perf_air_isentflow.stag_temp_from_mach(M0_pf, T0)
T0t_Mach_pf = perf_air_isentflow.stag_temp_from_vel(v0,T0)
T0t_T0_relation_pf = perf_air_isentflow.stagnation_static_rel(M0_pf)

# Semiperfect Gas
T0t_vel_spf = semiperf_air_isentflow.stag_temp_from_mach(M0_spf, T0)
T0t_Mach_spf = semiperf_air_isentflow.stag_temp_from_vel(v0,T0)
T0t_T0_relation_spf = semiperf_air_isentflow.stagnation_static_rel(M0_spf, T0)

print("Perfect Gas:       T0t/T0={0:8.6f};  T0t_vel={1:6.2f} K;  T0t_Mach={2:6.2f} K;  T0={3:6.2f} K".format(T0t_T0_relation_pf,
                                                                                                     T0t_vel_pf,
                                                                                                     T0t_Mach_pf,
                                                                                                     T0))

print("Semi-perfect Gas:  T0t/T0={0:8.6f};  T0t_vel={1:6.2f} K;  T0t_Mach={2:6.2f} K;  T0={3:6.2f} KK".format(T0t_T0_relation_spf,
                                                                                                     T0t_vel_spf,
                                                                                                     T0t_Mach_spf,
                                                                                                     T0))

Perfect Gas:       T0t/T0=1.077865;  T0t_vel=275.56 K;  T0t_Mach=275.56 K;  T0=255.65 K
Semi-perfect Gas:  T0t/T0=1.077981;  T0t_vel=275.59 K;  T0t_Mach=275.59 K;  T0=255.65 KK


The stagnation to static relation can be used to obtain the stagnation pressure, provided the static pressure (`stag_pressure_from_mach`):

$$ \frac{p_t}{p} = \left(\frac{T_t}{T}\right)^{\frac{\gamma}{\gamma-1}}$$

And similarly, the stagnation density (`stag_density_from_mach`):

$$ \frac{\rho_t}{\rho} = \left(\frac{T_t}{T}\right)^{\frac{1}{\gamma-1}}$$

In [7]:
# Perfect Gas:
p0t_pf = perf_air_isentflow.stag_pressure_from_mach(M0_pf, p0)
rho0t_pf = perf_air_isentflow.stag_density_from_mach(M0_pf, rho0)

# Semiperfect Gas
p0t_spf = semiperf_air_isentflow.stag_pressure_from_mach(M0_pf, p0, T0)
rho0t_spf = semiperf_air_isentflow.stag_density_from_mach(M0_pf, rho0, T0)

print("Perfect Gas:       p0t={0:6.0f} Pa;  p0={1:6.0f} Pa;  rho0t={2:8.6f} kg/m^3;  rho0={3:6.4f} kg/m^3".format(p0t_pf,
                                                                                                                  p0,
                                                                                                                  rho0t_pf,
                                                                                                                  rho0))

print("Semi-perfect Gas:  p0t={0:6.0f} Pa;  p0={1:6.0f} Pa;  rho0t={2:8.6f} kg/m^3;  rho0={3:6.4f} kg/m^3".format(p0t_spf,
                                                                                                                  p0,
                                                                                                                  rho0t_spf,
                                                                                                                  rho0))

Perfect Gas:       p0t= 70232 Pa;  p0= 54020 Pa;  rho0t=0.887903 kg/m^3;  rho0=0.7361 kg/m^3
Semi-perfect Gas:  p0t= 70241 Pa;  p0= 54020 Pa;  rho0t=0.887890 kg/m^3;  rho0=0.7361 kg/m^3


### Impact pressure, dynamic pressure and speed:

For obtaining the speed of the free stream, two options can be used:
- Obtaining the velocity from the stagnation and static temperatures (*vel_from_stag_temp*)
- Obatining the velocity from the Mach number (*vel_from_mach*)

In the first case, the stagnation and static temperatures must be kwown to gather the velocity:

$$ v = \sqrt{2c_p\left(T_t-T\right)}$$

While the velocity from the Mach numbr is obtained with:

$$v = aM=\sqrt{\gamma R_g T}M$$

In [8]:
# From stagnation temperature
v_TtT_pf = perf_air_isentflow.vel_from_stag_temp(T0t_Mach_pf, T0)
v_TtT_spf = semiperf_air_isentflow.vel_from_stag_temp(T0t_Mach_spf, T0)

# From Mach number:
v_M_pf = perf_air_isentflow.vel_from_mach(M0_pf, T0)
v_M_spf = semiperf_air_isentflow.vel_from_mach(M0_spf, T0)

print('Perfect Gas:       v_TtT={0:5.1f}m/s;   v_M={1:5.1f}m/s'.format(v_TtT_pf, v_M_spf))
print('Semi-perfect Gas:  v_TtT={0:5.1f}m/s;   v_M={1:5.1f}m/s'.format(v_TtT_spf, v_M_spf))

Perfect Gas:       v_TtT=200.0m/s;   v_M=200.0m/s
Semi-perfect Gas:  v_TtT=200.0m/s;   v_M=200.0m/s


If the flow is subsonic with $M<0.3$, the fluid may be considered *incompressible* (isochoric). In this case the total pressure can be calculated as:

$$p_t = p + q_i$$

Where $q_i$ is calculated with `dynamic_pressure_from_statpress`:
$$q_i = \frac{1}{2}\rho v^2$$

Or with `dynamic_pressure`:
$$q_i= \frac{\gamma}{2} p M^2$$

On the other hand, if the flow is considered *compressible* ($M>0.3$), the impact pressure can be calculated as the difference between the stagnation and static pressure (`impact_pressure_from_mach`):

$$q_i = p_t - p = p \left( \left(\frac{T_t}{T}\right)^{\frac{\gamma}{\gamma-1}}-1\right)$$


For example:

In [9]:
mach = 0.2 # -
vel = perf_air_isentflow.vel_from_mach(mach, T0)
T0t = perf_air_isentflow.stag_temp_from_mach(mach, T0)

qi_ls_p = perf_air_isentflow.dynamic_pressure_from_statpress(mach, p0)
qi_ls_rho = perf_air_isentflow.dynamic_pressure(vel, rho0)
qi_hs = perf_air_isentflow.impact_pressure_from_mach(M0_pf, p0, T0)

print('Dynamic pressure static pressure and Mach: M0={};  qi={}Pa'.format(mach, qi_ls_p))
print('Dynamic pressure from density and velocity: M0={};  qi={}Pa'.format(mach, qi_ls_rho))
print('Impact pressure: M0={};  qi={}Pa'.format(M0_pf, qi_hs))
print('(Impact pressure)/(static pressure)={0:6.4f}Pa;  (stagnation pressure)/(static pressure)={1:6.4f}Pa;'.format(qi_hs/p0, p0t_pf/p0))

Dynamic pressure static pressure and Mach: M0=0.2;  qi=1512.5317124605472Pa
Dynamic pressure from density and velocity: M0=0.2;  qi=1512.5317124605472Pa
Impact pressure: M0=0.6239758744916172;  qi=16211.817392755554Pa
(Impact pressure)/(static pressure)=0.3001Pa;  (stagnation pressure)/(static pressure)=1.3001Pa;
